In [1]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
import torch

raw_model = 'aiknowyou/mt5-base-it-paraphraser'

# Model and Tokenizer definition #
model = T5ForConditionalGeneration.from_pretrained(raw_model)
tokenizer = T5Tokenizer.from_pretrained(raw_model)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
max_size = 10000

def paraphrase(text, beams=100, grams=10, num_return_sequences=5):
    x = tokenizer(text, return_tensors='pt', padding=True).to(model.device)
    max_size = int(x.input_ids.shape[1] * 1.5 + 10)
    out = model.generate(**x, encoder_no_repeat_ngram_size=grams, num_beams=beams, num_return_sequences=num_return_sequences, max_length=max_size)
    return tokenizer.batch_decode(out, skip_special_tokens=True)
  
sentence = "Due amici si incontrano al bar per discutere del modo migliore di generare parafrasi."
print(paraphrase(sentence))

/Users/luciapassaro/virtual_environments/venv_multifakedetective/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['Due amici stanno discutendo del modo migliore per generare parafrasi.', 'Due amici si incontrano a un bar per discutere del modo migliore per generare parafrasi.', 'Due amici si incontrano al bar per parlare del modo migliore per generare parafrasi.', 'Due amici sono seduti al bar per discutere del modo migliore per generare parafrasi.', 'Due amici si incontrano in un bar per discutere del modo migliore per generare parafrasi.']


### Import (test) file

In [2]:
import json
import pandas as pd

f = open('test.json')
  
data = json.load(f)
  
df = pd.DataFrame.from_dict(data)
df

,id,date,link,title,content,statement_date,source,statement,verdict,verdict_ext,politician,political_party,piattaforma,politicians_in,macro_area,tags,links,versione
0,1061,2019-02-15,https://backend.pagellapolitica.it/dichiarazio...,Renzi sbaglia: la povertà non è diminuita con ...,[Il 14 febbraio l’ex presidente del Consiglio ...,14 febbraio 2019,https://www.corriere.it/sette/19_febbraio_14/r...,«Il tasso di povertà ha iniziato a diminuire q...,Falso,,Matteo Renzi,Italia Viva,Corriere della Sera,,economia,"[questioni sociali, occupazione, matteo renzi,...",[https://www.corriere.it/sette/19_febbraio_14/...,3
1,3262,2017-09-15,https://pagellapolitica.it/dichiarazioni/7897/...,Electric love,[Domenica 3 settembre il vicepresidente della ...,2017-09-03,https://www.facebook.com/LuigiDiMaio/videos/14...,«Un milione di auto elettriche in cinque anni ...,Nì,,Luigi Di Maio,,,,ambiente,"[nì, m5s, germania, luigi di maio, auto elettr...","[https://www.ambrosetti.eu/chi-siamo/, https:/...",4
2,56,2023-01-20T08:30:29,https://backend.pagellapolitica.it/dichiarazio...,Piantedosi dà i numeri sulla droga sequestrata...,"[Il 19 gennaio, ospite di Piazzapulita su La7,...",19 gennaio 2023,https://www.la7.it/piazzapulita/video/lintervi...,«L’anno scorso la Guardia di finanza ha fatto ...,Falso,[Nei primi nove mesi del 2022 in Italia sono s...,Matteo Piantedosi,,Piazzapulita – La7,,,"[piantedosi, sequestri, droga, italia]",[{'url': 'https://www.la7.it/piazzapulita/vide...,1
3,1628,2013-10-08,https://pagellapolitica.it/dichiarazioni/1975/...,"Il Porcellum! Lui ha parlato di Porcellum, che...","[Porcellum sì o Porcellum no?, La diatriba tra...",2013-09-30,http://www.ilsecoloxix.it/p/italia/2013/09/30/...,"""Il ""Porcellum""! Lui ha parlato di ""Porcellum""...",Nì,,Beppe Grillo,,,,istituzioni,"[nì, m5s, elezioni, istituzioni, legge elettor...",[https://pagellapolitica.it/dichiarazioni/anal...,4
4,114,2022-10-26T12:04:19,https://backend.pagellapolitica.it/dichiarazio...,È vero che Meloni alla Camera non ha parlato d...,"[Il 25 ottobre, il segretario del Partito demo...",25 ottobre 2022,https://twitter.com/EnricoLetta/status/1585004...,"«Giorgia Meloni non ha parlato di Sud, lotta a...",Falso,"[Nel suo discorso alla Camera, Meloni ha parla...",Enrico Letta,Partito Democratico,Twitter,,,"[critiche, letta, discorso, meloni, fiducia, c...",[{'url': 'https://twitter.com/EnricoLetta/stat...,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353,343,2021-10-01T13:04:59,https://backend.pagellapolitica.it/dichiarazio...,Con Draghi la spesa in sanità torna ai livelli...,"[Il 30 settembre, ospite a Omnibus su La7, il ...",30 settembre 2021,https://www.la7.it/omnibus/video/stefano-fassi...,«La nota di aggiornamento dell’agenda Draghi ...,Nì,,Stefano Fassina,Liberi e uguali,Omnibus – La7,,,"[con, draghi, la, spesa, in, sanita, torna, ai...",[{'url': 'https://www.la7.it/omnibus/video/ste...,3
354,3000,2015-11-29,https://pagellapolitica.it/dichiarazioni/6909/...,Salvini e il divieto ticinese del burqa,"[Ospite del programma Omnibus su La7, Matteo S...",2015-11-26,https://twitter.com/matteosalvinimi/status/669...,"""Parlamento del Canton Ticino ha messo FUORILE...",Vero,,Matteo Salvini,,,,esteri,"[questioni sociali, vero, lega nord, esteri]",[http://www3.ti.ch/CAN/comunicati/24-11-2015-c...,4
355,1229,2013-03-22,https://pagellapolitica.it/dichiarazioni/1145/...,Chi è che ha la maggioranza assoluta degli ita...,"[Il capogruppo al Senato del M5S, Vito Crimi, ...",2013-03-21,http://video.repubblica.it/dossier/movimento-5...,"""Chi è che ha la maggioranza assoluta degli it...",Nì,,Vito Crimi,,,,altro,"[nì, m5s, elezioni, istituzioni, capogruppo pa...",[http://elezioni.interno.it/camera/scrutini/20...,4
356,2506,2015-01-05,https://pagellapolitica.it/dichiarazioni/4836/...,"Uscire dall’euro si può, nonostante alcuni va ...",[Più che mai attuale in questi giorni in cui s...,2014-12-13,http://feedproxy.google.com/~r/beppegrillo/rss...,"""Uscire dall

### Paraphrase using mt5-base-it-paraphraser

In [3]:
paraphrases = []
for idx, claim in enumerate(list(df['statement'])):
    try:
        if idx%10 == 0:
            print(idx)
        paraphrases.append(paraphrase(claim))
    except:
        print(f'failed on {idx}: {claim}')
        paraphrases.append(['','','','',''])

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350


In [19]:
#df.rename(columns={"paraphrases": "paraphrases_mt5-base-it-paraphraser"})

df["paraphrases_mt5-base-it-paraphraser"] = paraphrases

### Write results (csv and json)

In [20]:
df.to_csv('test_paraphrases.csv',sep='\t',index=False)
result = df.to_json('test_paraphrases.json', orient='records')


f = open('test_paraphrases.json')
data = json.load(f)

dfp = pd.DataFrame.from_dict(data)


### see results (display column)

In [24]:
pd.set_option('display.max_colwidth', None)  
dfp[['statement','paraphrases_mt5-base-it-paraphraser']]


,statement,paraphrases_mt5-base-it-paraphraser
0,«Il tasso di povertà ha iniziato a diminuire quando ha iniziato a venir su la crescita»,"[Il tasso di povertà è aumentato quando ha iniziato a venire su la crescita, Il tasso di povertà è aumentato quando ha iniziato a venire su la crescita., Il tasso di povertà è aumentato quando ha iniziato a venire su la crescita, ma il tasso di povertà è aumentato quando ha iniziato a venire su la crescita., Il tasso di povertà è aumentato quando ha iniziato a venire su la crescita quando ha iniziato a venire su la crescita, Il tasso di povertà è aumentato quando ha iniziato a venire su la crescita; il tasso di povertà è aumentato quando ha iniziato a venire su la crescita.]"
1,"«Un milione di auto elettriche in cinque anni in Italia. Questo obiettivo se l’è dato la Germania al 2020, l’Olanda nel 2025 bandirà le auto a benzina e l’India lo farà nel 2030» (6’ 55’’)","[Questo obiettivo se l'è dato la Germania al 2020, l'Olanda nel 2025 bandirà auto a benzina e l'India lo farà nel 2030""., Questo obiettivo se l'è dato la Germania al 2020, l'Olanda nel 2025 bandirà auto a benzina e l'India lo farà nel 2030., Questo obiettivo se l'è dato la Germania al 2020, l'Olanda nel 2025 bandirà le auto elettriche e l'India lo farà nel 2030""., Questo obiettivo se l'è dato la Germania al 2020, l'Olanda nel 2025 bandirà le auto elettriche e l'India lo farà nel 2030., Questo obiettivo se l'è dato la Germania al 2020, l'Olanda nel 2025 bandirà le automobili a benzina e l'India lo farà nel 2030"".]"
2,«L’anno scorso la Guardia di finanza ha fatto complessivamente sequestri per 50 milioni di tonnellate di cocaina»,"[La Guardia di Finanza ha fatto complessivamente sequestri di cocaina per 50 milioni di tonnellate, L'anno scorso la Guardia di Finanza ha fatto complessivamente sequestri di 51 milioni di tonnellate di cocaina nel Regno Unito, L'anno scorso la Guardia di Finanza ha fatto complessivamente sequestri di 51 milioni di tonnellate di cocaina., L'anno scorso la Guardia di Finanza ha fatto complessivamente sequestri di 51 milioni di tonnellate di cocaina, La Guardia di Finanza ha fatto complessivamente sequestri di 51 milioni di tonnellate di cocaina nel Regno Unito]"
3,"""Il ""Porcellum""! Lui ha parlato di ""Porcellum"", che noi saremmo i responsabili e che vogliamo il ""Porcellum"". Siamo gli unici, Movimento 5 Stelle, che ha votato contro il ""Porcellum"" e il Signor Letta, ce l'ho qua, il documento andatevelo a vedere, ce l'ho qui il documento della firma elettronica di Letta che ha votato a favore del Porcellum, ha votato contro l'abrogazione del Porcellum!"".","[""Siamo gli unici, Movimento 5 stelle che ha votato contro il Porcellum, che noi saremmo responsabili e che vogliamo il Porcellum""., ""Siamo gli unici, Movimento 5 Stelle che ha votato contro il Porcellum, che noi saremmo responsabili e che vogliamo il Porcellum""., ""Siamo gli unici, Movimento 5 stelle, che ha votato contro il Porcellum, che noi saremmo responsabili e che vogliamo il Porcellum""., ""Il Signor Letta, che ha parlato di ""Porcellum"", ha parlato di ""Porcellum"", ha parlato di ""Porcellum"", ha parlato di ""Porcellum"", ha parlato di ""Porcellum"", ha parlato di ""Porcellum"", ha parlato di ""Porcellum"", ha parlato di ""Porcellum"", ha parlato di ""Porcellum"", ha parlato di ""Porcellum"", ha parlato di ""Porcellum"", ha parlato di ""Porcellum"", ha parlato di ""Porcellum"", ha parlato di ""Porcellum"", ha parlato di ""Porcellum"", ha parlato di ""Porcellum"", ha parlato di ""Porcellum"", ha parlato di ""Porcellum"", ha parlato di ""Porcellum"", ha parlato di ""Porcellum"", ha parlato di ""Porcellum"", ha parlato di ""Porcellum"", ha, ""Il Signor Letta, che ha parlato di ""Porcellum"", ha parlato di ""Porcellum"", ha parlato di ""Porcellum"", ha parlato di ""Porcellum"", ha parlato di ""Porcellum"", ha parlato di ""Porcellum"", ha parlato di ""Porcellum"", ha parlato di ""Porcellum"", ha parlato di ""Porcellum"", ha parlato di 